<a href="https://colab.research.google.com/github/Claire-Alais/Prototype-Insight/blob/main/Traitement_BD_IPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json import loads 
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
distances_paths_communes = pd.read_csv(r"distances_paths_communes.csp")

In [ ]:
dt = pd.read_csv(r"gdrive/My Drive/Insight Signals/Donnees/dt_base_habitants_idf.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dt_travail = pd.read_csv(r"gdrive/My Drive/Insight Signals/Donnees/dt_lieu_de_travail.csv")

In [ ]:
dt_nb_flux_dep = dt_travail[["CODGEO", "NBFLUX_C18_ACTOCC15P"]].groupby(by=["CODGEO"]).sum()

In [ ]:
float(dt_nb_flux_dep[dt_nb_flux_dep.index == 75101]["NBFLUX_C18_ACTOCC15P"])

8253.848717483848

In [ ]:
def divise(row):
  return  row.NBFLUX_C18_ACTOCC15P / float(dt_nb_flux_dep[dt_nb_flux_dep.index == row.CODGEO]["NBFLUX_C18_ACTOCC15P"])

In [ ]:
dt_travail["nbflux_percent"] = dt_travail.apply(divise, axis=1)

In [ ]:
list_dep = list(set(dt_travail.CODGEO))

In [ ]:
dict_list_dep = {dep: [] for dep in list_dep}
for dep in list_dep:
  for i in list(dt_travail[dt_travail.CODGEO == dep]["DCLT"]):
    for j in range(int(dt_travail[(dt_travail.CODGEO == dep) & (dt_travail.DCLT == i)]['NBFLUX_C18_ACTOCC15P'])):
        dict_list_dep[dep].append(i)

In [ ]:
dt = dt.drop(dt[dt.codgeo == 75056].index)

In [ ]:
import random

def lieu_de_travail (row):
  return(random.choice(dict_list_dep[row.codgeo]))

dt["codgeo_travail"] = dt.apply(lieu_de_travail, axis=1)

In [ ]:
dt.to_csv("dt_base_final.csv")
!cp dt_base_final.csv "gdrive/My Drive/Insight Signals/Donnees"

# check point

In [75]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json import loads 
from google.colab import drive
import random as rd

In [77]:
dt = pd.read_csv(r"gdrive/My Drive/Insight Signals/Donnees/dt_base_final.csv")

In [83]:
statistiques_mode = np.array([14683 / sum([14683, 9414, 834, 17126]),
                              834 / sum([14683, 9414, 834, 17126]),
                              17126 / sum([14683, 9414, 834, 17126]),
                              9414 / sum([14683, 9414, 834, 17126])])
#voiture, velo, marche, transport co

In [85]:
matrice_transport_duree = np.array([[0.58, 0.25, 0.13, 0.04],
                                    [0.64, 0.27, 0.06, 0.02],
                                    [0.84, 0.13, 0.03, 0.01],
                                    [0.08, 0.29, 0.41, 0.22]
          ])
#egt

In [91]:
matrice_duree_transport =  matrice_transport_duree * statistiques_mode

In [93]:
matrice_duree_transport.sum()

1.0881123237511

In [1]:
# on selectionne tous les individus qui travaillent
percentage_of_the_working_pop = 0.8

# on prend en compte le télétravail
percentage_of_homeworking = 0.2

percentage_of_agent_working = percentage_of_the_working_pop * (1 - percentage_of_homeworking)

In [2]:
def time_category (time_travel):
  if time_travel < 15:
    return 0
  if time_travel < 30:
    return 1
  if time_travel < 60:
    return 2
  return 3


def choice_of_a_mode_of_transport (row, include_common_transport=False):  
  category_car = time_category(row.time_travel_by_car)
  category_bike = time_category(row.time_travel_by_bike)
  category_walk = time_category(row.time_travel_by_walk)
  if include_common_transport:
    category_common_transport = time_category(row.time_travel_by_common_transport)

  chance_of_traveling_by_car = matrice_transport_duree[0, category_car]
  chance_of_traveling_by_bike = matrice_transport_duree[1, category_bike]
  chance_of_traveling_by_walk = matrice_transport_duree[2, category_walk]
  if include_common_transport:
    chance_of_traveling_by_common_transport = matrice_transport_duree[3, category_common_transport]

  tot_chance = chance_of_traveling_by_bike + chance_of_traveling_by_car + chance_of_traveling_by_walk
  if include_common_transport:
    tot_chance += chance_of_traveling_by_common_transport

  chance_of_traveling_by_car = chance_of_traveling_by_car / tot_chance
  chance_of_traveling_by_bike = chance_of_traveling_by_car + chance_of_traveling_by_bike / tot_chance
  chance_of_traveling_by_walk = chance_of_traveling_by_bike + chance_of_traveling_by_walk / tot_chance
  if include_common_transport:
    chance_of_traveling_by_common_transport = chance_of_traveling_by_walk + chance_of_traveling_by_common_transport / tot_chance

    return [chance_of_traveling_by_car, chance_of_traveling_by_bike, chance_of_traveling_by_walk, chance_of_traveling_by_common_transport]
  return [chance_of_traveling_by_car, chance_of_traveling_by_bike, chance_of_traveling_by_walk]

In [4]:
def multi_run_simulation(DataFrame, n=1000):
  for i in range(n):
    DataFrame["run_" + str(i + 1) + "_results"] = run_simulation(DataFrame["liste_of_chance"].values, np.random.uniform(0,1,len(DataFrame)))

    

def run_simulation(liste_of_chance, value):
  if value <= liste_of_chance[0]:
    return "car"
  if value <= liste_of_chance[1]:
    return "bike"
  if value <= liste_of_chance[2]:
    return "walk"
  if value <= liste_of_chance[3]:
    return "common transport"